# Crypto Portfolio Optimization

Goal: Create a machine learning-driven portfolio optimization system for cryptocurrencies, dynamically adjusting allocations to maximize returns and minimize risks based on predictive analytics.

### Top crypto cryptocurrencies 

In [159]:
import requests
import pandas as pd

##### Coingecko to get the top 30 cryptos 

In [160]:
#&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc

In [161]:
def fetch_top_coins_tickers(vs_currency='usd', limit=30):
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    params = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': limit,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '24h'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data: Status code {response.status_code}")
        return []

    data = response.json()

##Extract tickers
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

In [162]:
top_coins = fetch_top_coins_tickers(vs_currency='usd', limit=25)
print(top_coins)

['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA', 'AVAX', 'TRX', 'DOGE', 'LINK', 'DOT', 'MATIC', 'WBTC', 'TON', 'ICP', 'SHIB', 'UNI', 'BCH', 'LTC', 'DAI', 'IMX', 'FIL']


##### yfinance (historical data)

In [163]:
import yfinance as yf
import pandas as pd

# Specify the symbols for the cryptocurrencies you are interested in
symbols = [f"{coin}-USD" for coin in top_coins]  # Assuming 'top_coins' is a list of cryptocurrency symbols

# Initialize an empty DataFrame for concatenating all cryptocurrency data
df = pd.DataFrame()

# Specify the date range
start_date = '2021-01-01'
end_date = '2024-02-21'

# Fetch the historical data for each symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date)
    if not data.empty:
        data['Symbol'] = symbol.replace('-USD', '')  # Add a 'Symbol' column to identify the cryptocurrency
        df = pd.concat([all_data, data], axis=0)

# Reset the index of the concatenated DataFrame
df.reset_index(inplace=True)




[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for BTC-USD...
Fetching data for ETH-USD...
Fetching data for USDT-USD...
Fetching data for BNB-USD...
Fetching data for SOL-USD...
Fetching data for XRP-USD...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for STETH-USD...
Fetching data for USDC-USD...
Fetching data for ADA-USD...
Fetching data for AVAX-USD...
Fetching data for TRX-USD...
Fetching data for DOGE-USD...


...

Fetching data for LINK-USD...
Fetching data for DOT-USD...
Fetching data for MATIC-USD...
Fetching data for WBTC-USD...
Fetching data for TON-USD...
Fetching data for ICP-USD...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for SHIB-USD...
Fetching data for UNI-USD...
Fetching data for BCH-USD...
Fetching data for LTC-USD...
Fetching data for DAI-USD...
Fetching data for IMX-USD...



[*********************100%%**********************]  1 of 1 completed

Fetching data for FIL-USD...


In [164]:
all_data

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,BTC
1,2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,BTC
2,2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,BTC
3,2021-01-04,32810.949219,33440.218750,28722.755859,31971.914062,31971.914062,81163475344,BTC
4,2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,67547324782,BTC
...,...,...,...,...,...,...,...,...
2024-02-16 00:00:00,NaT,5.867093,5.985244,5.662748,5.795270,5.795270,197993160,FIL
2024-02-17 00:00:00,NaT,5.795997,6.377146,5.710954,6.199124,6.199124,456255634,FIL
2024-02-18 00:00:00,NaT,6.200620,6.421569,6.070881,6.275468,6.275468,318232526,FIL
2024-02-19 00:00:00,NaT,6.275813,7.770802,6.230459,7.427044,7.427044,797819922,FIL
